In [6]:
#Imports
import requests
import pandas as pd
import re
import json

In [7]:
URL = "https://marketplace.axieinfinity.com/axie?class=Aquatic&pureness=6&breedCount=0&breedCount=0"
resp = requests.get(URL)

In [8]:
resp.text

'<!DOCTYPE HTML>\n<html lang="en-US">\n<head>\n  <meta charset="UTF-8" />\n  <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n  <meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1" />\n  <meta name="robots" content="noindex, nofollow" />\n  <meta name="viewport" content="width=device-width,initial-scale=1" />\n  <title>Just a moment...</title>\n  <style type="text/css">\n    html, body {width: 100%; height: 100%; margin: 0; padding: 0;}\n    body {background-color: #ffffff; color: #000000; font-family:-apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", Roboto, Oxygen, Ubuntu, "Helvetica Neue",Arial, sans-serif; font-size: 16px; line-height: 1.7em;-webkit-font-smoothing: antialiased;}\n    h1 { text-align: center; font-weight:700; margin: 16px 0; font-size: 32px; color:#000000; line-height: 1.25;}\n    p {font-size: 20px; font-weight: 400; margin: 8px 0;}\n    p, .attribution, {text-align: center;}\n    #spinner {margin: 0 auto 30px auto; display: bl

In [9]:
def GeckoTT(ticker):
    # CoingeckoTickerTransformer
    listUrl = 'https://api.coingecko.com/api/v3/coins/list'
    idUrl = 'https://api.coingecko.com/api/v3/coins/'

    tokenList = requests.get(listUrl).json()
    
    for i in range(len(tokenList)):
        if tokenList[i]['symbol'] == ticker:
            tokenId = tokenList[i]['id']
            cgIdUrl = idUrl + tokenId

            tokenRes = requests.get(cgIdUrl)
            token    = tokenRes.json()
            name = token["name"].lower().replace(" ", "-")
            name = name.replace("binance-peg-", "")
            result = {"id":token["id"], "name":name}
            print(result)
            return result

In [13]:
def get_token_data(coin):
    # Formats
    coin_format       = GeckoTT(coin)
    coingecko_key     = coin_format["id"]
    coinmarketcap_key = coin_format["name"]
    
    # Coingecko data
    URL = f"https://api.coingecko.com/api/v3/coins/{coingecko_key}"
    resp = requests.get(URL).json()
    keys = ['symbol','name','sentiment_votes_up_percentage','sentiment_votes_down_percentage','market_cap_rank', 'coingecko_rank', 'coingecko_score', 'developer_score', 'community_score', 'liquidity_score', 'public_interest_score']
    values = [resp.get(key) for key in keys]

    res_coingecko  = pd.DataFrame({'indicator': keys, 'value': values  })
    social = pd.DataFrame({'indicator': resp["community_data"].keys(), 'value': resp["community_data"].values()  })
    
    # Coinmarketcap data
    URL = f"https://coinmarketcap.com/currencies/{coinmarketcap_key}/holders/"
    resp = requests.get(URL).text
    start_keyword, end_keyword  = '"holders":','"holderList":'
    before_keyword, start_keyword, result = resp.partition(start_keyword)
    result, end_keyword, after_keyword = result.partition(end_keyword)
    wallets =[x.group() for x in re.finditer(r'(?<=[:])([a-zA-Z0-9]*)*(?=[,])', result)]
    if len(wallets)==0: wallets = [None]
    if len(wallets)==1: wallets.append(None)
    if len(wallets) >2: wallets = [None,None]

    wallets_keys = ["holders","active wallets"]
    res_wallets = pd.DataFrame({'indicator': wallets_keys, 'value': wallets  })
    
    df = pd.concat([res_coingecko,social,res_wallets], ignore_index=True)
    return df

In [14]:
get_token_data("cspr")

{'id': 'casper-network', 'name': 'casper-network'}


,indicator,value
0,symbol,cspr
1,name,Casper Network
2,sentiment_votes_up_percentage,86.75
3,sentiment_votes_down_percentage,13.25
4,market_cap_rank,240
5,coingecko_rank,649
6,coingecko_score,26.384
7,developer_score,0.0
8,community_score,10.14
9,liquidity_score,51.575
